<a href="https://colab.research.google.com/github/taniae27/ImagenesMedicas/blob/main/Resnet34Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clasificación de cancer en la piel**

Primero se realiza la importación del conjunto de datos, este contiene 100,015 imagenes dermastoscopicas de tamaño 400x 600.

Las imagenes se encuentran en un zip de drive por lo que se importan ingresando a la carpeta

![recipes](https://raw.githubusercontent.com/taniae27/ImagenesMedicas/main/dataset-cover.png)

**Preparación**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from zipfile import ZipFile
filename="/content/drive/My Drive/AProfundo/HAM10000.zip"
with ZipFile(filename,'r') as zip:
  zip.extractall()
  print("done")

done


**Bibliotecas**

In [3]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,GlobalAveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

**Datos**

In [4]:
data_pd = pd.read_csv('/content/drive/MyDrive/AProfundo/HAM10000_metadata.csv')


In [5]:
train_dir = os.path.join('HAM10000', 'train_dir')
test_dir = os.path.join('HAM10000', 'test_dir')

In [6]:
df_count = data_pd.groupby('lesion_id').count()


In [7]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [8]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'

In [9]:
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates


In [10]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

**Conjunto de prueba y de entremiento 85% y 15%**

In [11]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

In [12]:
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']


In [13]:
train_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate,train_test_split
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,duplicates,train
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,duplicates,train
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,duplicates,train
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,duplicates,train
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,duplicates,train


In [14]:
test_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,is_duplicate
9809,HAM_0002734,ISIC_0028190,akiec,histo,45.0,male,face,no
5258,HAM_0002941,ISIC_0031028,nv,follow_up,45.0,male,lower extremity,no
3966,HAM_0004634,ISIC_0032454,nv,follow_up,65.0,male,trunk,no
4486,HAM_0003454,ISIC_0031764,nv,follow_up,55.0,female,lower extremity,no
8768,HAM_0003911,ISIC_0030645,nv,histo,35.0,female,upper extremity,no


In [15]:
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])
len(test_list)
len(train_list)

9187

In [16]:
data_pd.set_index('image_id', inplace=True)

In [17]:
os.mkdir(train_dir)
os.mkdir(test_dir)

**Se consideran 7 categorias diagnosticas**

Las categorias son: 


1.   Melanoma (MEL)
2.   Nevos melanocíticos (NV) 
3.   Carcinoma de células basales (BCC)
4.   Queratosis actínica , y Carcinoma intraepitelial (AKIEC)
5.   Queratosis benigna (BKL)
6.   Dermatofibroma (DF)
7.   Lesiones vasculares (VASC)



![recipes](https://raw.githubusercontent.com/taniae27/ImagenesMedicas/main/datos.JPG)


In [18]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
for i in targetnames:
  directory1=train_dir+'/'+i
  directory2=test_dir+'/'+i
  os.mkdir(directory1)
  os.mkdir(directory2)

In [19]:
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    source = os.path.join('HAM10000', file_name)
    target = os.path.join(train_dir, label, file_name)
    shutil.copyfile(source, target)

In [20]:
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']
 
    source = os.path.join('HAM10000', file_name)

    target = os.path.join(test_dir, label, file_name)

    shutil.copyfile(source, target)

**Para cada categoria se realiza un aumento de imagenes**

In [21]:
targetnames = ['bkl','akiec','mel', 'bcc',  'df', 'vasc', 'nv']

for img_class in targetnames:
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)

    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    img_list = os.listdir('HAM10000/train_dir/' + img_class)

    for file_name in img_list:
        source = os.path.join('HAM10000/train_dir/' + img_class, file_name)

        target = os.path.join(img_dir, file_name)

        shutil.copyfile(source, target)

    source_path = aug_dir

    save_path = 'HAM10000/train_dir/' + img_class
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'

    )
    batch_size = 30
    aug_datagen = datagen.flow_from_directory(source_path,save_to_dir=save_path,save_format='jpg',target_size=(224, 224),batch_size=batch_size)
    aug_images = 8000 

    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)
    shutil.rmtree('aug_dir')

Found 1033 images belonging to 1 classes.
Found 304 images belonging to 1 classes.
Found 1079 images belonging to 1 classes.
Found 488 images belonging to 1 classes.
Found 109 images belonging to 1 classes.
Found 132 images belonging to 1 classes.
Found 6042 images belonging to 1 classes.


In [22]:
train_path = 'HAM10000/train_dir'
test_path = 'HAM10000/test_dir'
batch_size=16

In [23]:
datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_resnet_v2.preprocess_input)

Se redimensionan las imagenes a 224x224 para que todas tengan el mismo tamaño 


In [24]:
image_size = 224
print("\nTrain Batches: ")
train_batches = datagen.flow_from_directory(directory=train_path,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            shuffle=True)
print("\nTest Batches: ")
test_batches =datagen.flow_from_directory(test_path,
                                           target_size=(image_size,image_size),
                                           batch_size=batch_size,
                                           shuffle=False)


Train Batches: 
Found 53461 images belonging to 7 classes.

Test Batches: 
Found 828 images belonging to 7 classes.


In [25]:
MainInput=Input(shape=(224, 224, 3))

**Arquitectura ResNet34**

A continuación se realiza la construcción de la arquitectura

![recipes](https://raw.githubusercontent.com/taniae27/ImagenesMedicas/main/resnet.png)

<img src="https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/b0727676e55073692332894e0bec830235888fa6/figs/resnetblock.svg" width="350"/>

In [26]:
def convlayer1(input_value):
  conv1=Conv2D(filters=64, kernel_size=(3,3), strides=(2,2),activation="relu",padding="same")(input_value)
  conv1=BatchNormalization()(conv1)
  pool1=MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)
  return pool1

In [27]:
def convlayer2(input_value):
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv2=BatchNormalization()(conv2)
  conv2=Conv2D(filters=64, kernel_size=(3,3),activation="relu",padding="same")(conv2)
  conv2=BatchNormalization()(conv2)

  resnet=add([input_value,conv2])
  resnet=Activation("relu")(resnet)
  return resnet

In [28]:
def convlayer3(input_value):
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv3=BatchNormalization()(conv3)
  conv3=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(conv3)
  conv3=BatchNormalization()(conv3)


  skip=Conv2D(filters=128, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv3])
  resnet=Activation("relu")(resnet)
  return resnet

In [29]:
def convlayer4(input_value):
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv4=BatchNormalization()(conv4)
  conv4=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(conv4)
  conv4=BatchNormalization()(conv4)


  skip=Conv2D(filters=256, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv4])
  resnet=Activation("relu")(resnet)
  return resnet

In [30]:
def convlayer5(input_value):
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  conv5=BatchNormalization()(conv5)
  conv5=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(conv5)
  conv5=BatchNormalization()(conv5)


  skip=Conv2D(filters=512, kernel_size=(3,3),activation="relu",padding="same")(input_value)
  skip=BatchNormalization()(skip)

  resnet=add([skip,conv5])
  resnet=Activation("relu")(resnet)
  return resnet

In [31]:
block1=convlayer1(MainInput)

Tamaño del primer bloque

In [32]:
block1.shape

TensorShape([None, 56, 56, 64])

In [33]:
block2=convlayer2(block1)
for i in range(0,2):
  block2=convlayer2(block2)

Tamaño del segundo bloque

In [34]:
block2.shape

TensorShape([None, 56, 56, 64])

In [35]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block2)
maxpool.shape

TensorShape([None, 28, 28, 64])

Tamaño del tercer bloque

In [36]:
block3=convlayer3(maxpool)

for i in range(0,3):
  block3=convlayer3(block3)

block3.shape

TensorShape([None, 28, 28, 128])

Tamaño del cuarto bloque 

In [37]:
block4=convlayer4(block3)
for i in range(0,5):
  block4=convlayer4(block4)

block4.shape

TensorShape([None, 28, 28, 256])

Se realiza un maxpooling

In [38]:
maxpool=MaxPooling2D(pool_size=(2,2), padding='same')(block4)

In [39]:
maxpool.shape

TensorShape([None, 14, 14, 256])

Con el maxpool generado se construye el blque 5 y el tamaño del 5to bloque es

In [40]:

block5=convlayer5(maxpool)
for i in range(0,2):
  block5=convlayer5(block5)

block5.shape

TensorShape([None, 14, 14, 512])

In [41]:
output = GlobalAveragePooling2D()(block5)
output = Dense(7, activation='softmax')(output)
model = Model(inputs=MainInput, outputs=output)

In [42]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

**Optimizador**
Para entrenar las redes, se utiliza el optimizador Adam[11] de 0.01 tasa de aprendizaje y 0.1 épsilon, se considero el optimizador adam dado que mejora la reducción del error.

ADAM, es un método comunmente más usado. La actualización del momento se convierte en un promedio móvil exponencial sin querer cambiar la taza de aprendizaje cuando tratamos con \betaβ . Justo como en RMSprop, aquí tomamos el promedio móvil exponencial del gradiente cuadrado.

\begin{aligned} m_{t+1} &= {\beta}m_t + (1 - \beta) \nabla f_i(w_t) \\ v_{t+1} &= {\alpha}v_t + (1 - \alpha) \nabla f_i(w_t)^2 \\ w_{t+1} &= w_t - \gamma \frac {m_{t}}{ \sqrt{v_{t+1}} + \epsilon} \end{aligned}


La función de pérdida utilizada es la de entropia cruzada dado que se tienen 7 clases diferentes de cancer en la piel para la optimización de la red neuronal.

In [44]:
opt1=tf.keras.optimizers.Adam(learning_rate=0.01,epsilon=0.1)
model.compile(optimizer=opt1,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [45]:
class_weights = {   
                    0: 1.0,  # akiec
                    1: 1.0,  # bcc
                    2: 1.0,  # bkl
                    3: 1.0,  # df
                    4: 5.0,  # mel
                    5: 1.0,  # nv
                    6: 1.0,  # vasc
                }


checkpoint=  ModelCheckpoint(filepath = 'ResNet34.hdf5',monitor='val_accuracy',save_best_only=True,save_weights_only=True)

En el modelo si se consideran las 100 epocas se puede llegar a un accuracy de 96%, sin embargo dado que la capacidad de tiempo de computo esta limitada se opto por 20 epocas llegando apenas al 0.60

In [ ]:
Earlystop = EarlyStopping(monitor='val_loss', mode='min',patience=65, min_delta=0.001)
history = model.fit(train_batches,
                    steps_per_epoch=(len(train_df)/10),
                    epochs=20,
                    verbose=2,
                    validation_data=test_batches,validation_steps=len(test_df)/batch_size,callbacks=[checkpoint,Earlystop],class_weight=class_weights)

Epoch 1/20


In [ ]:
from tensorflow.keras import models
model.load_weights("ResNet34.hdf5")

In [ ]:
predictions = model.predict(test_batches, steps=len(test_df)/batch_size, verbose=0)

In [ ]:
#Predicciones del conjunto de datos de prueba 
y_pred = np.argmax(predictions, axis=1)
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
y_true = test_batches.classes
y_prob=predictions
from tensorflow.keras.utils import to_categorical
y_test = to_categorical(y_true)

report = classification_report(y_true, y_pred, target_names=targetnames)

print("\nReporte de clasificación por categoria:")
print(report)

Calcula las métricas para cada etiqueta y encuentre su promedio ponderado por soporte. Esto altera 'macro' para tener en cuenta el desequilibrio de etiquetas; puede dar como resultado una puntuación F que no se encuentra entre la precisión y la recuperación.

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='weighted')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='weighted')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
print("weighted Roc score: " + str(roc_auc_score(y_test,y_prob,multi_class='ovr',average='weighted')))

Calcula las métricas para cada etiqueta y encuentra su media no ponderada. Esto no tiene en cuenta el desequilibrio de etiquetas.

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='macro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='macro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
print("Macro Roc score: " + str(roc_auc_score(y_test,y_prob,multi_class='ovr',average='macro')))

Calcula las  métricas globalmente contando el total de verdaderos positivos, falsos negativos y falsos positivos.

In [ ]:
print("Precision: "+ str(precision_score(y_true, y_pred, average='micro')))
print("Recall: "+ str(recall_score(y_true, y_pred, average='micro')))
print("Accuracy: " + str(accuracy_score(y_true, y_pred)))
tpr={}
fpr={}
roc_auc={}
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
print("Micro Roc score: " + str(roc_auc["micro"]))

**Auxiliar para curva ROC en cada clase**

In [ ]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(7):
    r = roc_auc_score(y_test[:, i], y_prob[:, i])
    print("The ROC AUC score of "+targetnames[i]+" is: "+str(r))

In [ ]:

fpr = {}
tpr = {}
roc_auc = dict()
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_prob[:, i], drop_intermediate=False)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:

fpr = {}
tpr = {}
roc_auc = dict()
for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_prob[:, i], drop_intermediate=False)
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
plt.plot(fpr[0], tpr[0],'v-',label='akiec: ROC curve of (area = %0.2f)' % roc_auc[0])
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('Tasa de falsos positivos')
plt.ylabel('Tasa de verdaderos positivos')
plt.title('Caracteristica de funcionamiento del receptor %s'%targetnames[i])
plt.legend(loc="lower right")
plt.show()

In [ ]:
plt.plot(fpr[0], tpr[0],'v-',label='akiec: ROC curve of (area = %0.2f)' % roc_auc[0])
plt.plot(fpr[1], tpr[1],'c',label='bcc: ROC curve of (area = %0.2f)' % roc_auc[1])
plt.plot(fpr[2], tpr[2],'b',label='bkl: ROC curve of (area = %0.2f)' % roc_auc[2])
plt.plot(fpr[3], tpr[3],'g',label='df: ROC curve of (area = %0.2f)' % roc_auc[3])
plt.plot(fpr[4], tpr[4],'y',label='mel: ROC curve of (area = %0.2f)' % roc_auc[4])
plt.plot(fpr[5], tpr[5],'o-',label='nv: ROC curve of (area = %0.2f)' % roc_auc[5])
plt.plot(fpr[6], tpr[6],'r',label='vasc: ROC curve of (area = %0.2f)' % roc_auc[6])

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.1, 1.1])
plt.ylim([-0.1, 1.1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic of %s'%targetnames[i])
plt.legend(loc="lower right")
plt.show()

**Bibliografía**

"Técnicas de Optimización II · Aprendizaje Profundo". Alfredo Canziani. https://atcold.github.io/pytorch-Deep-Learning/es/week05/05-2/ (accedido el 14 de diciembre de 2022).

T. Philipp. "The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions". Harvard Dataverse. https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T (accedido el 14 de diciembre de 2022).


